# Basic scraping for Guardian discussions u

In [47]:
! ./.g


[master 7182c5e] web
 6 files changed, 942 insertions(+), 1446 deletions(-)
 create mode 100644 gn7h4.pkl
 rewrite guardian comments scraper.ipynb (84%)
 create mode 100644 h2tzt.pkl
 create mode 100644 h452h.pkl
 create mode 100644 hvg6j.pkl
 create mode 100644 hvpq8.pkl
Enumerating objects: 10, done.
Counting objects: 100% (10/10), done.
Delta compression using up to 4 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 1.63 MiB | 79.00 KiB/s, done.
Total 8 (delta 5), reused 0 (delta 0)
fatal: the remote end hung up unexpectedly
fatal: the remote end hung up unexpectedly
Everything up-to-date


In [59]:
with open('guardian comments scraper.ipynb','r') as fp:
    nb = fp.read()

In [60]:
import re

pp = re.compile("(https://www.the.*?)'")
lks = pp.findall(nb)

with open('links.txt','a') as fp:
    fp.write('\n'.join(lks))

In [1]:
import re 
import requests
import pickle
import json

In [34]:
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/19/ditch-the-algorithm-generation-students-a-levels-politics'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/21/joe-biden-democrat-convention-nice-guy-trump-fight'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/23/the-cruel-exams-algorithm-has-laid-bare-the-unfairness-at-the-heart-of-our-schools'
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/01/us-mainstream-right-vigilante-terror'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/31/politicians-covid-19-statistics-statisticians'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/14/boris-johnson-is-telling-scotland-that-the-union-is-no-longer-based-on-consent'

In [71]:
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/29/rival-scientists-lockdowns-scientific-covid-19'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/08/johnson-and-sturgeon-will-need-a-better-philosophy-than-precaution-to-rid-us-of-covid'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/10/continual-local-lockdowns-answer-covid-control'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/11/the-rebel-scientists-cause-would-be-more-persuasive-if-it-werent-so-half-baked'

article_url = 'https://www.theguardian.com/commentisfree/2021/apr/03/undermining-the-astrazeneca-jab-is-a-dangerous-act-of-political-folly#comment-148440261'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/09/boris-johnson-brexit-belfast-violence-eu-good-friday-agreement'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/02/culture-war-government-race-report'
article_url = 'https://www.theguardian.com/commentisfree/2021/mar/18/britain-sunny-vaccine-optimism-april-delivery-delay'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/14/boris-johnson-is-telling-scotland-that-the-union-is-no-longer-based-on-consent'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/15/britain-harness-social-sciences-covid-pandemic-deprivation'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/16/government-race-report-evidence#comment-148662066'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/20/britain-falling-birthrate-covid-pandemic-conservatives-removed-support-for-parents'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/21/bmi-eating-disorder-pandemic-treatment'

In [36]:

key = my_keys.keys['guardian']

NameError: name 'my_keys' is not defined

In [72]:
#scrape the page and get the short url
short_url_pp =  re.compile('"shortUrlId":"(.*?)"')

r = requests.get(article_url)
mm = short_url_pp.search(r.text)
short_url = mm.group(1)

#the short url allows us to access the discussion
comments_url = 'https://api.nextgen.guardianapps.co.uk/discussion/%s.json'%short_url
params = {'orderBy' : 'newest',
          'pageSize' : 25,
          'displayThreaded' : 'true',
          'commentsClosed' : 'false',
          'page' : '',
          'maxResponses' : 50
         }

json_data =  []
start_page  = 1
last_page = None


for num_page in range(start_page, 10**4):
    print('getting comments page no.', num_page)
    params['page'] = num_page
    r = requests.get( comments_url, params=params)
    json_data.append( r.text)
    #The last page is a field in the json object
    #I don't need to read this as json I could do a regexp
    if num_page == start_page:
        json_dict = json.loads(r.text)
        last_page = json_dict['lastPage']
        print('there are %d pages'%last_page)
    if num_page == last_page : break
        

fn = '%s.pkl'%short_url.split('/')[-1]
print('archiving to %s'%fn)
with open(fn,'wb') as fp:
    pickle.dump(json_data,fp)
    


getting comments page no. 1
there are 4 pages
getting comments page no. 2
getting comments page no. 3
getting comments page no. 4
archiving to h6v39.pkl


## remember each page of comments has been stored separately

In [16]:
dd = json.loads(json_data[0])
len(json_data)

11

In [17]:
xx = json.loads(json_data[0])
xx.keys()

dict_keys(['postedCommentHtml', 'commentsHtml', 'commentCount', 'refreshStatus', 'lastPage', 'paginationHtml'])

In [136]:
print( xx['commentsHtml'][200:700])

 class="d-comment
            d-comment--top-level"
           id="comment-144422422"
           data-comment-id="144422422"
           data-comment-author="Hushing"
           data-comment-replies="0"
           data-comment-timestamp="2020-10-11T14:25:06.000+01:00"
           data-comment-author-id="12407494"
           data-comment-highlighted="false"
           itemscope itemtype="http://schema.org/Comment">

    <div class="d-comment__inner d-comment__inner--top-level">
        <div class="


In [66]:
pa = re.compile('data-comment-author="(.*?)"')
pid = re.compile('data-comment-author-id="(\d+)')
pcid = re.compile('\sid="comment-(.*?)"')
pp = re.compile('<li class="d-comment(.*?)>',re.DOTALL)

auths = []
md = []
for xx in json_data:
    hh = json.loads(xx)['commentsHtml']
    auths.extend(list( zip(pa.findall(hh), 
                           pid.findall(hh)
                          )))
    md.extend(pp.findall(hh))

auths = list(set(auths))
auths.sort(key=lambda x : x[0].lower())


In [102]:
#print( '\n'.join([x for x in md if 'pinkie' in x.lower()]))

https://discussion.theguardian.com/comment-permalink/144375739

In [8]:
auths = dict(auths)

In [9]:
comments_url

'https://api.nextgen.guardianapps.co.uk/discussion//p/hvpq8.json'

In [73]:
class Comment():
    pass

from bs4 import BeautifulSoup

def html2comments(x):
    tt = json.loads(x)
    soup = BeautifulSoup(tt['commentsHtml'],  "lxml")
    raw = [ comment.text for comment in  
            soup.findAll('div', {'class' : "d-comment__body"}) ]
    
    metadata = soup.findAll('span', {'class' : "d-comment__author"})
    
    
    auth = [ x.text.upper().strip() for x in metadata]
    auth_id = [ x.find('a')['href'] for x in metadata]
    #this was a pain to do really
    #the problem is blocked comments
    comment_id = [ x['id'][len('comment-'):] for x in soup.findAll('li')]
 
    
    ss = '##{}\nhttps://discussion.theguardian.com/comment-permalink/{}\n{}\n\n{}'
    return [ ss.format(a,b,d,c.strip())  
                   for a,b,d,c in zip(auth, comment_id, auth_id, raw)]
     
all_texts = [ html2comments(page) for page in json_data[:] ]



from itertools import chain
all_comments = list(chain.from_iterable(all_texts))
    

# Random data hacking

In [74]:
A = [x.split('\n')[0] for x in all_comments]
from collections import Counter
AN = Counter(A)
sorted([(x,y) for x,y in AN.items() if y > 4], key=lambda x : -x[1]) 

[('##FRAZZLEDSTUDENTNURSE', 16),
 ('##ONONA', 11),
 ('##CLARITYOFTHOUGHT', 9),
 ('##LISAMARIE3', 7),
 ('##MJBESWICK', 6),
 ('##MATTHEWHENSON', 6),
 ('##VANLEEUW', 5),
 ('##RICHIE34', 5),
 ('##BLUEJAY2011', 5)]

In [149]:
! ./.g


[master b8c8a75] web
 2 files changed, 1457 insertions(+), 1317 deletions(-)
 rewrite guardian comments scraper.ipynb (76%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 47.51 KiB | 84.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/macbuse/GUARDIAN.git
   d207214..b8c8a75  master -> master


In [52]:
print(all_comments[0])

##NELLIESWEET
https://discussion.theguardian.com/comment-permalink/148657977
https://profile.theguardian.com/user/id/105310467

...so there are common social rules about vaccination. It is a universal social obligation.I don't think I've ever thought about protecting other people when I've had a vaccine. Screw them. It was either because I wanted to go somewhere which wouldn't let me in without one, or, like with Covid, it was pure self-interest. I'll have the Covid vaccine as soon as they come to me, but only because I don't want to catch the damn thing.


In [75]:
all_comments.sort(key=lambda x : -len(x) )
print( '\n\n'.join(all_comments[:20]) )

##ANDYPANDY1968
https://discussion.theguardian.com/comment-permalink/148776512
https://profile.theguardian.com/user/id/12992447

I didn't specifically know BMI was used as a crude measure to access NHS eating disorder services, but it doesn't surprise me. 
I think the cold fact is that NHS eating disorder services are indeed hopelessly underfunded as indeed are NHS mental health services in general. Given this is the case, the BMI measure itself is not really the issue. The issue is a lack of resource in this area, and as usual, anything in the realm of mental health has the added problem of stigma. There is also a lack of 'joining the dots' in terms of the eventual cost of physical ill health caused by the different types of eating disorder. 
Malnutrition is not just too little but also too much intake, or any unhealthy intake.
I myself was somewhat overweight for many years. I didn't really think of the mental health connection, but with hindsight, I now realise it was an element to 

In [18]:
#https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

import nltk
#nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gregmcshane/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
a = '''My views haven't changed. Leaving the EU will be of no benefit to anyone except those who hold money and power along with certain corporates and a smidgen of medium to small businesses.
How else can it be when you are shutting shop to the most developed nations, the richest nations in the world, who are either in the EU or have market agreements with the EU. Even the emerging markets of China and India along with other developing economies are aspiring to develop trading agreements with the EU, the second largest economy in the world. Any deal with the UK will be secondary and will depend upon their agreement with the EU.
Boris Johnson once said "f**k business". I take this as meaning there is something in it for them and they don't really care if businesses find it difficulA
t to cope.
I can see ministers and conservative MPs queuing to fill their pockets by acting as advisors for potential trade deals whilst the rest of the country are left struggling trying to rebuild their lives, especially following the pandemic. The same ministers will probably be looking to lower all kinds of standards from food and welfare to safety at work to improve their opportunities.
When it comes to brinkmanship, I can only view this as getting away with whatever they can. I do not believe it is anything to do with negotiating an agreement that will be to the benefit of the UK or the EU as a whole, I don't believe our ministers are capable of doing that.'''

In [24]:
sid.polarity_scores(a)
sid.sentiment_valence(a)

TypeError: sentiment_valence() missing 4 required positional arguments: 'sentitext', 'item', 'i', and 'sentiments'

In [70]:
target = 'decolon'
target = 'science'
target = 'immigration'


xx = [x for x in all_comments if target.lower() in x.lower() and  len(x) > 20]
yy = [x for x in all_comments if '#pin' in x.lower() and 2000 > len(x) > 20]

#print('\n\n'.join( ['\n\n'.join(x) for x in zip(xx,yy)])  )


#print( '\n\n'.join(xx).replace(target, ) )
pxx = re.compile(target, re.I)
print( pxx.sub('**' + target.upper() + '**', 
              '\n\n'.join(xx)) )
               
#print( '\n\n'.join(xx) )


##FRED1
https://discussion.theguardian.com/comment-permalink/148740973
https://profile.theguardian.com/user/id/2979908

What's the point in moaning about population numbers either way?
It's not like we can really do anything about it. Population growth or decline depends on the independent decisions of billions of people and the wonders of modern medicine. It is whatever it is.
Sure we can educate, we can incentivise, we can disincentivise, but we're talking billions of variables. 
Population is a problem. Doing anything about it is not a solution. Dealing with it is the solution. We need solutions to the population that we have not the one we wished we had.
Some countries have ageing populations. Some have young populations. There's a solution. It's not a straightforward solution. Many cities are under pressure already with the populations they have so we do need better infrastructure.
But the bigger issue to overcome is just how damn popular populist anti-**IMMIGRATION** politicians 

36

In [64]:
23**2

529

## 